In [3]:
from sklearn import svm
from skopt.space import Real
from skopt.utils import use_named_args
from skimage.feature import hog
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
import csv
from skopt import gp_minimize
import cv2
import concurrent.futures
from os import listdir
from os.path import isfile, join
import numpy as np

path = '../data'

# Categorías
categorias = ['NORMAL', 'PNEUMONIA']

espacio_svm = [
    Real(0.01, 1000, prior='log-uniform', name='C'),
    Real(0.0001, 1, prior='log-uniform', name='gamma'),
]

def procesar_imagen(imagen_path, hog_params, size, etiqueta):
    imagen = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        print(f'No se pudo leer la imagen: {imagen_path}')
        return None, None

    imagen = cv2.resize(imagen, size)
    fd = hog(imagen, orientations=hog_params['orientations'], pixels_per_cell=hog_params['pixels_per_cell'],
             cells_per_block=(1,1), visualize=False)

    return fd, etiqueta

def cargar_datos(ruta, hog_params, size=(128, 128)):
    datos = []
    etiquetas = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i, cat in enumerate(categorias):
            carpeta = join(ruta, cat)
            archivos = [f for f in listdir(carpeta) if isfile(join(carpeta, f))]
            for archivo in archivos:
                imagen_path = join(carpeta, archivo)
                futures.append(executor.submit(procesar_imagen, imagen_path, hog_params, size, i))

        for future in concurrent.futures.as_completed(futures):
            fd, label = future.result()
            if fd is not None:
                datos.append(fd)
                etiquetas.append(label)

    if len(datos) == 0 or len(etiquetas) == 0:
        raise ValueError("No se pudieron cargar los datos. Asegúrate de que la ruta y las categorías son correctas.")

    return np.array(datos), np.array(etiquetas)

# Nombre del archivo CSV para los resultados
nombre_archivo_csv = "../results/models_parameter_combination/svm_combination_with_hog.csv"
# Inicializar el archivo CSV
with open(nombre_archivo_csv, 'w', newline='') as archivo:
    escritor = csv.writer(archivo)
    escritor.writerow(['C', 'Gamma', 'Accuracy', 'Recall', 'Specificity', 'Precision', 'F1-Score', 'AUC'])

@use_named_args(espacio_svm)
def funcion_objetivo_svm(C, gamma):
    hog_params = {'orientations': 9, 'pixels_per_cell': (16, 16)} # Mejor combinación encontrada para HOG con SVM
    datos_entrenamiento, etiquetas_entrenamiento = cargar_datos(join(path, 'train'), hog_params)
    datos_prueba, etiquetas_prueba = cargar_datos(join(path, 'test'), hog_params)
    modelo = svm.SVC(C=C, gamma=gamma, random_state=42, probability=True)
    modelo.fit(datos_entrenamiento, etiquetas_entrenamiento)
    etiquetas_predichas = modelo.predict(datos_prueba)
    accuracy = accuracy_score(etiquetas_prueba, etiquetas_predichas)
    f1 = f1_score(etiquetas_prueba, etiquetas_predichas)
    precision = precision_score(etiquetas_prueba, etiquetas_predichas)
    recall = recall_score(etiquetas_prueba, etiquetas_predichas)
    cm = confusion_matrix(etiquetas_prueba, etiquetas_predichas)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1])
    auc = roc_auc_score(etiquetas_prueba, modelo.predict_proba(datos_prueba)[:, 1])
    
    # Registrar en CSV
    with open(nombre_archivo_csv, 'a', newline='') as archivo:
        escritor = csv.writer(archivo)
        escritor.writerow([C, gamma, accuracy, recall, specificity, precision, f1, auc])
    
    return -f1  # El objetivo sigue siendo maximizar el F1-Score

resultado_svm = gp_minimize(funcion_objetivo_svm, espacio_svm, n_calls=20, random_state=42, n_jobs=-1, verbose=True)

# Imprimir el mejor resultado para SVM
print(f"Mejor F1-Score: {-resultado_svm.fun}")
print(f"Mejores hiperparámetros SVM: C={resultado_svm.x[0]}, gamma={resultado_svm.x[1]}")

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 58.4115
Function value obtained: -0.8324
Current minimum: -0.8324
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 63.0275
Function value obtained: -0.8386
Current minimum: -0.8386
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 125.9457
Function value obtained: -0.7692
Current minimum: -0.8386
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 121.6126
Function value obtained: -0.8193
Current minimum: -0.8386
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 141.4372
Function value obtained: -0.8008
Current minimum: -0.8386
Iteration No: 6 s